In [4]:
import test

weekLen, weekNum, prep, minWork, maxWork, pause, minWeek, maxWeek, early, latest, startDay, startMonth, startYear, zone, allExercise, needs = test.prepare_input("ex1.xlsx")
print("weekLen: ", weekLen)
print("weekNum: ", weekNum)
print("prep: ", prep)
print("minWork: ", minWork)
print("maxWork: ", maxWork)
print("pause: ", pause)
print("minWeek: ", minWeek)
print("maxWeek: ", maxWeek)
print("early: ", early)
print("latest: ", latest)
print("startDay: ", startDay)
print("startMonth: ", startMonth)
print("startYear: ", startYear)
print("zone: ", zone)
print(f"allExercise {len(allExercise)}: {allExercise}")
print("needs: ", needs)


weekLen:  7
weekNum:  3
prep:  25
minWork:  45
maxWork:  100
pause:  5
minWeek:  2
maxWeek:  4
early:  8
latest:  22
startDay:  20
startMonth:  7
startYear:  2023
zone:  2
allExercise 46: [{'name': 'Cycling Warmup', 'category': 'warm up', 'setTime': 7, 'priority': 4, 'biceps': -1, 'chest': -1, 'triceps': -1, 'shoulder': -1, 'abdominal': -1, 'backUp': -1, 'backLow': -1, 'thighUp': 0, 'thighLow': 0, 'calves': 0, 'glutes': -1}, {'name': 'Rowing Warmup', 'category': 'warm up', 'setTime': 7, 'priority': 4, 'biceps': 0, 'chest': -1, 'triceps': -1, 'shoulder': -1, 'abdominal': -1, 'backUp': 0, 'backLow': 0, 'thighUp': -1, 'thighLow': -1, 'calves': -1, 'glutes': -1}, {'name': 'Running Warmup', 'category': 'warm up', 'setTime': 7, 'priority': 4, 'biceps': -1, 'chest': -1, 'triceps': -1, 'shoulder': -1, 'abdominal': -1, 'backUp': -1, 'backLow': -1, 'thighUp': 0, 'thighLow': 0, 'calves': 0, 'glutes': 0}, {'name': 'Abduction Machine', 'category': 'resistance', 'setTime': 6, 'priority': 3, 'biceps'

In [22]:
import icalendar

def load_calendar(cpath):
    file = open(cpath)
    cal = icalendar.Calendar.from_ical(file.read())
    events = []
    for component in cal.walk():
        if component.name == "VEVENT":
            events.append({
                "event": component.decoded("summary"), 
                "start": component.decoded("dtstart"), 
                "end": component.decoded("dtend"),
                "stamp": component.decoded("dtstamp") # can be ignored, not relevant for task
                })
        else:

            print(f"{component}")
    file.close()
    return events

c = load_calendar("cal1.ics")
len(c)

VCALENDAR({'VERSION': vText('b'2.0''), 'PRODID': vText('b'-//trainmORe calendar//'')}, VEVENT({'SUMMARY': vText('b'Heuristic Optimization: Exercise''), 'DTSTART': <icalendar.prop.vDDDTypes object at 0x136454e20>, 'DTEND': <icalendar.prop.vDDDTypes object at 0x136454ca0>, 'DTSTAMP': <icalendar.prop.vDDDTypes object at 0x136454520>}), VEVENT({'SUMMARY': vText('b'Heuristic Optimization: Lecture''), 'DTSTART': <icalendar.prop.vDDDTypes object at 0x136454f10>, 'DTEND': <icalendar.prop.vDDDTypes object at 0x136454160>, 'DTSTAMP': <icalendar.prop.vDDDTypes object at 0x136454d00>}), VEVENT({'SUMMARY': vText('b'Column Generation und Branch-and-Price: Lecture''), 'DTSTART': <icalendar.prop.vDDDTypes object at 0x136454970>, 'DTEND': <icalendar.prop.vDDDTypes object at 0x136454490>, 'DTSTAMP': <icalendar.prop.vDDDTypes object at 0x136454eb0>}), VEVENT({'SUMMARY': vText('b'Lunch Break''), 'DTSTART': <icalendar.prop.vDDDTypes object at 0x136454640>, 'DTEND': <icalendar.prop.vDDDTypes object at 0x136

110

In [ ]:
import gurobipy as gp
def solve(full_instance_path, calendar_path):
    pass